In [15]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import shutil
import string
!pip install tensorflow
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/58/70/e8ac764ec80810eefcbab0cb1d21dbba6cf26719c44cd6d9a5e9f0407935/tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.python

  Obtaining dependency information for tensorboard-data-server<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/73/c6/825dab04195756cf8ff2e12698f22513b3db2f64925bdd41671bfb33aaa5/tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:00:00:0100:02
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached gast-0.5.4-py3-none-any.whl (19 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Using cached grpcio-1.64.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
Using cached h5py-3.11.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
Using cached libclang-18.1.1-

2024-06-18 19:56:28.883471: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 19:56:28.888604: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 19:56:28.952453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 19:56:29.800592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
to_train = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset.csv')
data = to_train.copy()
priority = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_priority.csv')
to_train.head()

,Issue_id,Priority,Title,Description,Component,Combined_Text,Topic
0,1,P3,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...,Team,"['Usability', 'issue', 'external', 'editors', ...",5
1,13,P3,CC Discussion: local versioning (1GAT3PL),Would it make sense to have a (the?) projects ...,Team,"['CC', 'Discussion', 'local', 'versioning', '1...",5
2,12,P3,Manage/unmanage support and policies (1GALAEG),Currently every new resource added to a shared...,Team,"['Manage/unmanage', 'support', 'policies', '1G...",5
3,11,P3,API: ISharingManager::load mapping vcm project...,Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...,Team,"['API', 'ISharingManager', ':load', 'mapping',...",5
4,10,P3,API - VCM event notification (1G8G6RR),There seems to be a need for a VCM Event notif...,Team,"['API', 'VCM', 'event', 'notification', '1G8G6...",5


In [6]:
priority_subsection = to_train[['Issue_id','Priority']].copy()

In [8]:
extract_dir = '/home/rpierson/PiersonREU/extracted'
train_priority_path = os.path.join(extract_dir, 'train_priority.csv')
priority_subsection.to_csv(train_priority_path, index = False)

In [10]:
data = data.drop(columns=['Priority'])
data_path = os.path.join(extract_dir, 'train_dataset.csv')
data.to_csv(data_path, index = False)

In [11]:
priority_subsection.head()

,Issue_id,Priority
0,1,P3
1,13,P3
2,12,P3
3,11,P3
4,10,P3


In [12]:
data.head()

,Issue_id,Title,Description,Component,Combined_Text,Topic
0,1,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...,Team,"['Usability', 'issue', 'external', 'editors', ...",5
1,13,CC Discussion: local versioning (1GAT3PL),Would it make sense to have a (the?) projects ...,Team,"['CC', 'Discussion', 'local', 'versioning', '1...",5
2,12,Manage/unmanage support and policies (1GALAEG),Currently every new resource added to a shared...,Team,"['Manage/unmanage', 'support', 'policies', '1G...",5
3,11,API: ISharingManager::load mapping vcm project...,Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...,Team,"['API', 'ISharingManager', ':load', 'mapping',...",5
4,10,API - VCM event notification (1G8G6RR),There seems to be a need for a VCM Event notif...,Team,"['API', 'VCM', 'event', 'notification', '1G8G6...",5


In [16]:
data_zero = data[data['Topic']==0]

In [17]:
data_zero

,Issue_id,Title,Description,Component,Combined_Text,Topic
177,146,UnknownHostException: Repositories (1GEWRWG),UnknownHostException is not caught when the ho...,Team,"['UnknownHostException', 'Repositories', '1GEW...",0
194,234,NPE in sync (1GJ4W10),Did a sync of Eclipse UI. It came up in the c...,Team,"['NPE', 'sync', '1GJ4W10', 'sync', 'Eclipse', ...",0
246,217,UI BUG TO BE FILED (1GHQOGF),While minimizing an Eclipse running in Eclipse...,Team,"['UI', 'BUG', 'FILED', '1GHQOGF', 'minimizing'...",0
337,1749,Debugger did not show source for type; and gen...,There are two problems here:; problem 1:; - ra...,UI,"['Debugger', 'show', 'source', 'type', 'genera...",0
348,1811,Scripted Handler Example does not run (1GENWSM),scripted handler example does not run in 116; ...,Scripting,"['Scripted', 'Handler', 'Example', 'run', '1GE...",0
...,...,...,...,...,...,...
68028,229022,[Progress] Display#getShells returns disposed ...,I20080427-2000; ; To reproduce the latest I-Bu...,UI,"['Progress', 'Display', 'getShells', 'returns'...",0
68032,229097,NPE when using OpenCheatSheetFromHelpAction,Created attachment 97790; Demo project; ; Buil...,User Assistance,"['NPE', 'using', 'OpenCheatSheetFromHelpAction...",0
68079,229473,[Contributions] ShowInMenu dispose is attempti...,I20080429-0100; ; I got this by quitting Eclip...,UI,"['Contributions', 'ShowInMenu', 'dispose', 'at...",0
68108,229712,NPE in MenuServiceFactory likely from File->Re...,I found two of these in my .log. I see restar...,UI,"['NPE', 'MenuServiceFactory', 'likely', 'File-...",0


In [18]:
data_zero_text = data_zero[['Combined_Text']]

In [19]:
data_zero_text

,Combined_Text
177,"['UnknownHostException', 'Repositories', '1GEW..."
194,"['NPE', 'sync', '1GJ4W10', 'sync', 'Eclipse', ..."
246,"['UI', 'BUG', 'FILED', '1GHQOGF', 'minimizing'..."
337,"['Debugger', 'show', 'source', 'type', 'genera..."
348,"['Scripted', 'Handler', 'Example', 'run', '1GE..."
...,...
68028,"['Progress', 'Display', 'getShells', 'returns'..."
68032,"['NPE', 'using', 'OpenCheatSheetFromHelpAction..."
68079,"['Contributions', 'ShowInMenu', 'dispose', 'at..."
68108,"['NPE', 'MenuServiceFactory', 'likely', 'File-..."
